## import data

In [42]:
Cancer <- read.table('./Cancer-Data.dat',header = F)
Colname <- c('ID', 'E_AGE','CANCER','DYR1B','BIOPSY','INFLAM',
             'FAD','FCD','INPAP','OTHER','FC_CYS','FC_APM',
             'FC_ADEN','FC_SCAD','FC_EHPP','FC_INFL','FC_CALC',
             'AGEMENA','MCHILD','AGEDX','HORMX1','FAMHIS','YRFU',
             'TIME','AGEBCH','AGMENOP','HORMSF','AGEHR','YRHR') 
colnames(Cancer) <- Colname

In [57]:
head(Cancer)
Cancer[] <- lapply(Cancer, as.numeric)
table(Cancer$CANCER)

ID,E_AGE,CANCER,DYR1B,BIOPSY,INFLAM,FAD,FCD,INPAP,OTHER,⋯,AGEDX,HORMX1,FAMHIS,YRFU,TIME,AGEBCH,AGMENOP,HORMSF,AGEHR,YRHR
4,76,0,62,1,-1,-1,-1,-1,-1,⋯,43,0,0,96,398,23,45,0,0,0
21,63,0,63,1,0,0,0,0,0,⋯,30,1,0,96,398,0,-1,0,0,0
47,62,0,68,1,0,0,1,0,0,⋯,34,0,1,96,336,21,36,0,0,0
98,58,1,65,2,0,0,1,0,0,⋯,37,0,0,86,257,18,52,1,56,85
110,42,0,65,2,1,0,1,1,0,⋯,29,1,1,78,159,-1,38,-1,-1,-1
133,48,1,65,2,0,0,1,0,0,⋯,33,0,0,80,187,-1,0,0,0,0



  0   1 
270  66 

### xgboost

In [51]:
source('./lib/train.R')
source('./lib/test.R')
source('./lib/cross_validation.R')

In [56]:
dat_train <- Cancer[,-c(1,3)]

label_train <- as.numeric(Cancer$CANCER)
head(label_train)
Cancer_cv <- cv(dat_train,
                label_train,
                run.xgboost = T,
                run.gbm = F, 
                run.adaboost = F,
                K = 5,
                par = NULL,
                num_class = 2)
Cancer_cv

[1] 0 0 0 1 0 1

[1] "current iteration is 1"
[1] "current iteration is 2"
[1] "current iteration is 3"
[1] "current iteration is 4"
[1] "current iteration is 5"


$error
[1] 0.08928885

$sd
[1] 0.02792903

### logistic regression

In [58]:
names(Cancer)

[1] "ID"      "E_AGE"   "CANCER"  "DYR1B"   "BIOPSY"  "INFLAM"  "FAD"    
 [8] "FCD"     "INPAP"   "OTHER"   "FC_CYS"  "FC_APM"  "FC_ADEN" "FC_SCAD"
[15] "FC_EHPP" "FC_INFL" "FC_CALC" "AGEMENA" "MCHILD"  "AGEDX"   "HORMX1" 
[22] "FAMHIS"  "YRFU"    "TIME"    "AGEBCH"  "AGMENOP" "HORMSF"  "AGEHR"  
[29] "YRHR"

In [63]:
glm.cancer.full <- glm(CANCER ~ .,
              family = binomial("logit"), data = Cancer)
summary(glm.cancer.full)


Call:
glm(formula = CANCER ~ ., family = binomial("logit"), data = Cancer)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2554  -0.6438  -0.4152  -0.1647   3.1833  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) 21.5362359  7.6805771   2.804  0.00505 ** 
ID           0.0018432  0.0008464   2.178  0.02942 *  
E_AGE       -0.4125970  0.5652732  -0.730  0.46545    
DYR1B       -0.2515807  0.4016904  -0.626  0.53111    
BIOPSY       0.1979577  0.2560906   0.773  0.43952    
INFLAM       0.4548603  0.4422057   1.029  0.30366    
FAD          0.7324692  0.4408637   1.661  0.09662 .  
FCD          1.4404262  0.7732156   1.863  0.06248 .  
INPAP       -0.8879261  0.8581296  -1.035  0.30080    
OTHER       -0.9964963  0.7645246  -1.303  0.19243    
FC_CYS      -0.2761295  0.3384623  -0.816  0.41459    
FC_APM       0.2118043  0.3837732   0.552  0.58102    
FC_ADEN      0.0005598  0.2762389   0.002  0.99838    
FC_SCAD     -0.0160248  0

In [71]:
glm.cancer.fit1 <- glm(CANCER ~ ID + FAD + FCD + 
                       AGEMENA + HORMSF + YRHR,
                       family = binomial("logit"), data = Cancer)
summary(glm.cancer.fit1)


Call:
glm(formula = CANCER ~ ID + FAD + FCD + AGEMENA + HORMSF + YRHR, 
    family = binomial("logit"), data = Cancer)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5980  -0.6792  -0.5359  -0.2552   2.9030  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.1182111  0.5915826  -0.200 0.841620    
ID          -0.0002308  0.0001350  -1.710 0.087228 .  
FAD          0.4084523  0.3824227   1.068 0.285491    
FCD          1.1878334  0.3924469   3.027 0.002472 ** 
AGEMENA     -0.1305471  0.0368969  -3.538 0.000403 ***
HORMSF       2.8262643  0.5998863   4.711 2.46e-06 ***
YRHR        -0.0366803  0.0083017  -4.418 9.94e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 332.92  on 335  degrees of freedom
Residual deviance: 291.94  on 329  degrees of freedom
AIC: 305.94

Number of Fisher Scoring iterations: 5


In [72]:
step(glm.cancer.fit1,direction = "backward")$anova

Start:  AIC=305.94
CANCER ~ ID + FAD + FCD + AGEMENA + HORMSF + YRHR

          Df Deviance    AIC
- FAD      1   293.07 305.07
<none>         291.94 305.94
- ID       1   294.88 306.88
- FCD      1   304.71 316.71
- AGEMENA  1   305.00 317.00
- YRHR     1   313.84 325.84
- HORMSF   1   316.90 328.90

Step:  AIC=305.07
CANCER ~ ID + FCD + AGEMENA + HORMSF + YRHR

          Df Deviance    AIC
<none>         293.07 305.07
- ID       1   296.09 306.09
- AGEMENA  1   306.13 316.13
- FCD      1   306.24 316.24
- YRHR     1   315.06 325.06
- HORMSF   1   317.93 327.93


Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,329,291.9359,305.9359
- FAD,1,1.129154,330,293.0650,305.0650


In [73]:
glm.cancer.fit2 <- glm(CANCER ~ ID + FCD + 
                       AGEMENA + HORMSF + YRHR,
                       family = binomial("logit"), data = Cancer)
summary(glm.cancer.fit2)


Call:
glm(formula = CANCER ~ ID + FCD + AGEMENA + HORMSF + YRHR, family = binomial("logit"), 
    data = Cancer)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.6209  -0.6779  -0.5284  -0.2735   2.7201  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.0270670  0.5673107  -0.048 0.961947    
ID          -0.0002335  0.0001348  -1.732 0.083354 .  
FCD          1.1508750  0.3725897   3.089 0.002009 ** 
AGEMENA     -0.1299898  0.0366914  -3.543 0.000396 ***
HORMSF       2.8105556  0.5974862   4.704 2.55e-06 ***
YRHR        -0.0366423  0.0082763  -4.427 9.54e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 332.92  on 335  degrees of freedom
Residual deviance: 293.07  on 330  degrees of freedom
AIC: 305.07

Number of Fisher Scoring iterations: 5


In [74]:
step(glm.cancer.fit2,direction = "backward")$anova

Start:  AIC=305.07
CANCER ~ ID + FCD + AGEMENA + HORMSF + YRHR

          Df Deviance    AIC
<none>         293.07 305.07
- ID       1   296.09 306.09
- AGEMENA  1   306.13 316.13
- FCD      1   306.24 316.24
- YRHR     1   315.06 325.06
- HORMSF   1   317.93 327.93


Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,330,293.065,305.065


In [75]:
glm.cancer.fit3 <- glm(CANCER ~ FCD + AGEMENA +
                       HORMSF + YRHR,
                       family = binomial("logit"), data = Cancer)
summary(glm.cancer.fit3)


Call:
glm(formula = CANCER ~ FCD + AGEMENA + HORMSF + YRHR, family = binomial("logit"), 
    data = Cancer)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5456  -0.7246  -0.5388  -0.3071   2.7140  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.475997   0.502332  -0.948 0.343346    
FCD          1.173916   0.376879   3.115 0.001840 ** 
AGEMENA     -0.135797   0.036170  -3.754 0.000174 ***
HORMSF       2.725811   0.593202   4.595 4.33e-06 ***
YRHR        -0.035677   0.008189  -4.357 1.32e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 332.92  on 335  degrees of freedom
Residual deviance: 296.09  on 331  degrees of freedom
AIC: 306.09

Number of Fisher Scoring iterations: 5


## import data2

In [82]:
crime <- read.table('APPENC02(1).txt')
head(crime)
dim(crime)

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17
1,Los_Angeles,CA,4060,8863164,32.1,9.7,23677,27700,688936,70.0,22.3,11.6,8.0,20786,184230,4
2,Cook,IL,946,5105067,29.2,12.4,15153,21550,436936,73.4,22.8,11.1,7.2,21729,110928,2
3,Harris,TX,1729,2818199,31.3,7.1,7553,12449,253526,74.9,25.4,12.5,5.7,19517,55003,3
4,San_Diego,CA,4205,2498016,33.5,10.9,5905,6179,173821,81.9,25.3,8.1,6.1,19588,48931,4
5,Orange,CA,790,2410556,32.6,9.2,6062,6369,144524,81.2,27.8,5.2,4.8,24400,58818,4
6,Kings,NY,71,2300664,28.3,12.4,4861,8942,680966,63.7,16.6,19.5,9.5,16803,38658,1


[1] 440  17

In [85]:
lm.crime.full <- lm(V10 ~ V4 + V5 + V6 +
                    V7 + V8 + V9 + V10 + V11 + V12 + V13 + 
                    V14 + V15 + V16 + I(V17), data = crime)
summary(lm.crime.full)

Warning message in model.matrix.default(mt, mf, contrasts):
“the response appeared on the right-hand side and was dropped”Warning message in model.matrix.default(mt, mf, contrasts):
“problem with term 7 in model.matrix: no columns are assigned”


Call:
lm(formula = V10 ~ V4 + V5 + V6 + V7 + V8 + V9 + V10 + V11 + 
    V12 + V13 + V14 + V15 + V16 + I(V17), data = crime)

Residuals:
   Min     1Q Median     3Q    Max 
-74484  -5753    326   5069 407568 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -5.093e+04  3.534e+04  -1.441 0.150362    
V4          -3.054e+00  8.485e-01  -3.599 0.000357 ***
V5           2.343e-01  2.027e-02  11.560  < 2e-16 ***
V6           2.211e+02  4.247e+02   0.521 0.602962    
V7           3.212e+01  3.806e+02   0.084 0.932790    
V8          -5.189e+00  3.150e+00  -1.647 0.100229    
V9           3.404e+00  2.280e+00   1.493 0.136294    
V11         -2.656e+02  3.218e+02  -0.825 0.409688    
V12          1.409e+02  3.735e+02   0.377 0.706154    
V13          1.143e+03  4.881e+02   2.341 0.019693 *  
V14         -1.597e+02  6.580e+02  -0.243 0.808403    
V15          2.335e+00  6.993e-01   3.339 0.000913 ***
V16         -7.070e+00  9.459e-01  -7.475 4.44e-13 ***
I(V17)

In [86]:
lm.crime.fit1 <- lm(V10 ~ V4 + V5 + V13 + 
                    V15 + V16, data = crime)
summary(lm.crime.fit1)


Call:
lm(formula = V10 ~ V4 + V5 + V13 + V15 + V16, data = crime)

Residuals:
   Min     1Q Median     3Q    Max 
-77382  -5773    285   5117 406547 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -6.389e+04  1.023e+04  -6.244 1.02e-09 ***
V4          -3.109e+00  7.580e-01  -4.101 4.91e-05 ***
V5           2.496e-01  1.633e-02  15.282  < 2e-16 ***
V13          1.450e+03  3.071e+02   4.721 3.18e-06 ***
V15          2.460e+00  4.685e-01   5.250 2.39e-07 ***
V16         -7.899e+00  7.870e-01 -10.037  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 23270 on 434 degrees of freedom
Multiple R-squared:  0.8421,	Adjusted R-squared:  0.8403 
F-statistic: 462.9 on 5 and 434 DF,  p-value: < 2.2e-16


In [87]:
step(lm.crime.fit1,direction = "backward")$anova

Start:  AIC=8854.47
V10 ~ V4 + V5 + V13 + V15 + V16

       Df  Sum of Sq        RSS    AIC
<none>               2.3511e+11 8854.5
- V4    1 9.1118e+09 2.4422e+11 8869.2
- V13   1 1.2072e+10 2.4718e+11 8874.5
- V15   1 1.4929e+10 2.5004e+11 8879.6
- V16   1 5.4571e+10 2.8968e+11 8944.3
- V5    1 1.2651e+11 3.6161e+11 9041.9


Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,434,235106929833,8854.474


In [ ]:
crime <- 